# 循环神经网络—TensorFlow

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import matplotlib.pyplot as plt

## 导入 MNIST 数据集

In [2]:
mnist = input_data.read_data_sets('./datasets/ch08/tensorflow/MNIST',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./datasets/ch08/tensorflow/MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./datasets/ch08/tensorflow/MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./datasets/ch08/tensorflow/MNIST\t10k-images-idx3-ubyte.gz
Extracting ./datasets/ch08/tensorflow/MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(55000, 784)
(55000, 10)


In [4]:
print(mnist.validation.images.shape)
print(mnist.validation.labels.shape)

(5000, 784)
(5000, 10)


In [5]:
print(mnist.test.images.shape)
print(mnist.test.labels.shape)

(10000, 784)
(10000, 10)


## 定义占位符 placeholder

In [6]:
# 参数设置
batch_size = 100        # BATCH 的大小，相当于一次处理100个image
time_step = 28          # 一个LSTM中，输入序列的长度Tx，image有28行
input_size = 28         # 单个x向量长度，image有28列
lr = 0.001              # 学习率
num_units = 100         # 隐藏层多少个LTSM单元
iterations =1000        # 迭代训练次数
classes =10             # 输出大小，0-9十个数字的概率

# 定义 placeholders
# 维度是[batch_size，time_step * input_size]
x = tf.placeholder(tf.float32, [None, time_step * input_size]) 
# 输入的是二维数据，将其还原为三维，维度是[batch_size, time_step, input_size]
x_image = tf.reshape(x, [-1, time_step, input_size])                   
y = tf.placeholder(tf.int32, [None, classes])                     

## 定义RNN（LSTM）结构

In [7]:
rnn_cell = tf.contrib.rnn.BasicLSTMCell(num_units=num_units) 
outputs,final_state = tf.nn.dynamic_rnn(
    cell=rnn_cell,              # 选择传入的cell
    inputs=x_image,             # 传入的数据
    initial_state=None,         # 初始状态
    dtype=tf.float32,           # 数据类型
    time_major=False,           # False: (batch, time_step, input); True: (time_step, batch, input)，这里根据x_image结构选择False
)
output = tf.layers.dense(inputs=outputs[:, -1, :], units=classes)    

## 定义损失函数与优化算法

In [8]:
cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=output)      # 计算loss
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)      #选择优化方法

In [9]:
#判断预测标签和实际标签是否匹配
correct_prediction = tf.equal(tf.argmax(y, axis=1),tf.argmax(output, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))  #计算正确率

## 训练并验证准确率

In [10]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y})
    if (i+1) % 50 == 0:
        print("train accuracy %.3f" % accuracy.eval(session = sess,
                feed_dict = {x:batch_x, y:batch_y}))
print("test accuracy %.3f" % accuracy.eval(session = sess,
    feed_dict = {x:mnist.test.images, y:mnist.test.labels}))

train accuracy 0.560
train accuracy 0.740
train accuracy 0.900
train accuracy 0.810
train accuracy 0.940
train accuracy 0.890
train accuracy 0.880
train accuracy 0.910
train accuracy 0.910
train accuracy 0.930
train accuracy 0.870
train accuracy 0.960
train accuracy 0.960
train accuracy 0.940
train accuracy 0.930
train accuracy 0.960
train accuracy 0.940
train accuracy 0.980
train accuracy 0.920
train accuracy 0.980
test accuracy 0.958
